In [7]:
import sys
sys.path.append('../')
from postprocessing import post_processing


# read test data
test_dir = '../data/processed/census_income_test.csv'
print("\nTest data: ###############################")
X_test, y_test = post_processing(test_dir, get_dummies=True, over_18=True)
X_test = X_test.astype(int)
print(f'\nX_test shape: {X_test.shape}')


Test data: ###############################
Number of numeric columns: 8
Original shape: (72023, 28)
Binary shape: (72023, 93)

X_test shape: (72023, 82)


In [10]:
import pickle
# read model
with open('../ML/lgbn_v3.pkl', 'rb') as file:
    model = pickle.load(file)
print("model was read successfully")

model was read successfully


In [23]:
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from dash_bootstrap_components.themes import FLATLY
import numpy as np
import types

# read model
with open('../ML/lgbn_v3.pkl', 'rb') as file:
    model = pickle.load(file)
print("model was read successfully")

# Select a random
#  1000 sample from X_test and y_test
random_indices = np.random.choice(X_test.index, size=1000, replace=False)
X_test_sample = X_test.loc[random_indices]
y_test_sample = y_test.loc[random_indices]

# Create a wrapper class for the LightGBM model
class ModelWrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, X):
        return self.model.predict(X)

    def predict_proba(self, X):
        pred = self.model.predict(X)
        return np.array([1 - pred, pred]).T

# Wrap the LightGBM model
wrapped_model = ModelWrapper(model)

# Create the explainer
explainer = ClassifierExplainer(wrapped_model,
                                X_test_sample,
                                y_test_sample,
                                labels=['<=50K', '>50K'],
                                target='target')

# Disable some settings to expedite the process
db = ExplainerDashboard(explainer,
                   mode='external',
                   importances=True,
                   shap_interaction=False,
                   decision_trees=False,
                   whatif= False,
                   simple= False,
                   contributions= True,
                   bootstrap=FLATLY)


model was read successfully
Note: shap values for shap='kernel' normally get calculated against X_background, but paramater X_background=None, so setting X_background=shap.sample(X, 50)...
Generating self.shap_explainer = shap.KernelExplainer(model, X, link='identity')
Building ExplainerDashboard..
Generating layout...
Calculating shap values...


e:\DS\dku_practice\.venv\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



  0%|          | 0/1000 [00:00<?, ?it/s]

Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating pred_percentiles...
Calculating predictions...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...


In [24]:
# export yaml
db.to_yaml("dashboard.yaml", explainerfile="explainer.joblib", dump_explainer=True)

Dumping configuration .yaml to e:\DS\dku_practice\webapp\dashboard.yaml...
Dumping explainer to e:\DS\dku_practice\webapp\explainer.joblib...
Warning! KernelExplainer does not work with joblib or pickle, but only with dill, so specify e.g. filepath='explainer.dill' to use dill instead of joblib or pickle.


PicklingError: Can't pickle <function ClassifierExplainer.shap_explainer.<locals>.model_predict at 0x0000024B57710AE0>: it's not found as explainerdashboard.explainers.ClassifierExplainer.shap_explainer.<locals>.model_predict